# 18 • Herramientas de texto

En este notebook se revisará distintas herramientas para modificar, limpiar, preprocesar texto en Python.

## Contenido
1. Strings como lista de caracteres
2. Preproceso de texto
3. Regular expressions (*Regex*)
4. spaCy
5. Referencias  

## 1. Strings como lista de caracteres
Anteriormente ya habían trabajado con strings, sin embargo, podemos tratar a un string como una lista de caracteres. Para esto, podemos utilizar los [métodos para *strings* que tiene Python](https://docs.python.org/3/library/stdtypes.html#string-methods).

In [96]:
texto = "Mi viLLano faVorito"

# También se puede declarar así:
# texto = str("Mi villano favorito")

In [97]:
# Un string es una cadena de caracteres y al imprimirlo con un loop regresa un carácter por renglón
for i in texto:
    print(i)

M
i
 
v
i
L
L
a
n
o
 
f
a
V
o
r
i
t
o


Existen algunas funciones propias de los *strings* que nos ayudarán para trabajar con textos, como:
- **lower**, pone la cadena en minúsculas
- **upper**, pone la cadena en mayúsculas
- **title**, pone en mayúscula la primera letra de cada palabra de un enunciado
- **replace**, para sustituir algún texto
- **rfind**, encuentra dentro de una cadena dónde inicia un texto de nuestro interés
- **rspilt**, separa un enunciado en palabras (*tokens*)

In [98]:
# lower
texto.lower()

'mi villano favorito'

In [99]:
# upper
texto.upper()

'MI VILLANO FAVORITO'

In [100]:
# title
texto.title()

'Mi Villano Favorito'

In [101]:
# replace
texto.replace("viLLano", "marciano")

'Mi marciano faVorito'

In [102]:
# rfind
texto.rfind("panda") # no está este string, entonces devuelve -1

-1

In [103]:
# rfind
texto.rfind("viLLano") # sí está este string, entonces devuelve >0

3

In [104]:
# split
texto.lower().rsplit()

['mi', 'villano', 'favorito']

## 2. Preproceso de texto
Usualmente la información en formato de texto se encuentra desordenada y sin estructura. Para poder realizar un mejor análisis es importante normalizar esta información.

Existen distintas herramientas para el preproceso de datos que podemos utilizar para obtener una base de datos limpia, entre las que se encuentran: utilizar letras minúscula, separa un enunciado en *tokens*, lematización (usar la raíz de la palabra), remover puntuación y *stopwords*, entre otras.

In [105]:
# load libraries
import pandas as pd
import numpy as np

### Base de datos inicial (raw)

Usaremos la base de datos de **IMDB Dataset of 50K Movie Reviews (Spanish)** publicada en Kaggle, para poder correr este archivo deben de bajar previamente la base, la cual tiene por nombre de "IMDB Dataset SPANISH.csv"

Liga: https://www.kaggle.com/datasets/luisdiegofv97/imdb-dataset-of-50k-movie-reviews-spanish


In [106]:
%%time

# obtener información
column_names = ("username","date","tweet")
df = pd.read_csv('../../IMDB Dataset SPANISH.csv') # <- HAY QUE ADECUAR DE DONDE JALAN LA BASE DE DATOS DESPUES QUE LA BAJEN DE KAGGLE
df = df.drop(columns=["Unnamed: 0", "review_en", "sentiment", "sentimiento"]).rename(columns={"review_es":"reseña"})

CPU times: user 1.74 s, sys: 444 ms, total: 2.18 s
Wall time: 2.21 s


In [108]:
# así se ve la base de datos
df

,reseña
0,Uno de los otros críticos ha mencionado que de...
1,Una pequeña pequeña producción.La técnica de f...
2,Pensé que esta era una manera maravillosa de p...
3,"Básicamente, hay una familia donde un niño peq..."
4,"El ""amor en el tiempo"" de Petter Mattei es una..."
...,...
49995,Pensé que esta película hizo un buen trabajo a...
49996,"Mala parcela, mal diálogo, mala actuación, dir..."
49997,Soy católica enseñada en escuelas primarias pa...
49998,Voy a tener que estar en desacuerdo con el com...


In [109]:
# descripción general de los datos
df.describe()

,reseña
count,50000
unique,49599
top,"Hilarante, limpio, alegre y digno de cita.¿Qué..."
freq,4


In [110]:
# identificación de valores únicos por variable
df.nunique()

reseña    49599
dtype: int64

### Eliminar signos de puntuación
Aquí se eliminan los signos de puntuación con excepción de `#` para poder identificar los hashtags.

In [112]:
#library that contains punctuation
import string
signos_puntuacion = string.punctuation
signos_puntuacion = signos_puntuacion.replace("#", "")  # quitar '#' de hashtags 
signos_puntuacion

'!"$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [113]:
#defining the function to remove punctuation
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in signos_puntuacion])
    return punctuationfree

#storing the puntuation free text
df['reseña_limpia'] = df['reseña'].apply(lambda x:remove_punctuation(x))
df.head()

,reseña,reseña_limpia
0,Uno de los otros críticos ha mencionado que de...,Uno de los otros críticos ha mencionado que de...
1,Una pequeña pequeña producción.La técnica de f...,Una pequeña pequeña producciónLa técnica de fi...
2,Pensé que esta era una manera maravillosa de p...,Pensé que esta era una manera maravillosa de p...
3,"Básicamente, hay una familia donde un niño peq...",Básicamente hay una familia donde un niño pequ...
4,"El ""amor en el tiempo"" de Petter Mattei es una...",El amor en el tiempo de Petter Mattei es una p...


### Textos en minúsculas
Parte de la normalización es poner todos los textos en minúsculas (o mayúsculas), esto puede servir posteriormente por ejemplo para hacer un conteo de palabras.

⚠️ _**Nota**: Al hacer esto existe el riesgo que se pierda información contextual; por ejemplo, dentro de una reseña cuando alguien escribe una palabra específica en mayúsculas y las demás en minúsculas esto conlleva un mayor importancia o aumento en tono de voz._

In [114]:
# cambiar los tweets a letras minúsculas
df['reseña_limpia'] = df['reseña_limpia'].apply((lambda x: x.lower()))
df.head()

,reseña,reseña_limpia
0,Uno de los otros críticos ha mencionado que de...,uno de los otros críticos ha mencionado que de...
1,Una pequeña pequeña producción.La técnica de f...,una pequeña pequeña producciónla técnica de fi...
2,Pensé que esta era una manera maravillosa de p...,pensé que esta era una manera maravillosa de p...
3,"Básicamente, hay una familia donde un niño peq...",básicamente hay una familia donde un niño pequ...
4,"El ""amor en el tiempo"" de Petter Mattei es una...",el amor en el tiempo de petter mattei es una p...


### Tokenization
Aquí veremos cómo separar los enunciados en *tokens* (palabras o cadenas de caracteres).

In [115]:
# getting the tokens per tweet
df['reseña_limpia'] = df['reseña_limpia'].apply(lambda x: x.rsplit())
df.head()

,reseña,reseña_limpia
0,Uno de los otros críticos ha mencionado que de...,"[uno, de, los, otros, críticos, ha, mencionado..."
1,Una pequeña pequeña producción.La técnica de f...,"[una, pequeña, pequeña, producciónla, técnica,..."
2,Pensé que esta era una manera maravillosa de p...,"[pensé, que, esta, era, una, manera, maravillo..."
3,"Básicamente, hay una familia donde un niño peq...","[básicamente, hay, una, familia, donde, un, ni..."
4,"El ""amor en el tiempo"" de Petter Mattei es una...","[el, amor, en, el, tiempo, de, petter, mattei,..."


### Eliminar *stopwords*
Las *stopwords* son palabras de uso común que no añaden un valor adicional al análisis de texto o que no tienen sentido, por lo que pueden ser eliminadas de nuestra lista de tokens.

Existen distintas maneras de obtener la lista de stopwords:  
> (1) Crear nuestra propia lista  

> (2) Buscar alguna base de datos que contenga la lista de stopwords
> - En GitHub encontré una lista de stopwords el el repositorio [Alir3z4/stop-words](https://github.com/Alir3z4/stop-words/blob/master/spanish.txt)
> - Una alternativa es una lista de Kaggle [Spanish Stopwords W2V](https://www.kaggle.com/code/mpwolke/spanish-stopwords-w2v)
> - Otra alternativa es la lista que da la página [countwordsfree.com](https://countwordsfree.com/stopwords/spanish)

> (3) Algo más estandarizado y completo es utilizar la librería de *Natural Language Toolkit* (**NLTK**), la cual tiene listas de *stopwords* en distintos idiomas, donde (además de inglés) podemos encontrar la lista de palabras en español. Para instalar este paquete así como mayor detalle de uso, revisa la página de la [librería NLTK](https://www.nltk.org/data.html).

⚠️ _**Nota**: antes de usar alguna de las opciones anteriores deberás de pensar cuál lista de stopwords se acopla mejor a tus necesidades!_

In [117]:
# (1) Crear nuestra propia lista de stopwords
stopwords_1 = ['a','ante','bajo','con','no','de','del','al','tras','idiota','bitcoin']

print("Número de stopwords en mi lista (1):", len(stopwords_1))
stopwords_1

Número de stopwords en mi lista (1): 11


['a',
 'ante',
 'bajo',
 'con',
 'no',
 'de',
 'del',
 'al',
 'tras',
 'idiota',
 'bitcoin']

In [118]:
# (2) Buscar alguna base de datos que contenga la lista de stopwords.
#     En este ejemplo bajamos la lista de GitHub
import urllib3

http = urllib3.PoolManager()
r = http.request('GET', "https://raw.githubusercontent.com/Alir3z4/stop-words/master/spanish.txt")
stopwords_2 = r.data.decode('utf-8').replace("\n"," ").rsplit()

print("Número de stopwords en mi lista (2):", len(stopwords_2))
stopwords_2[:15]

Número de stopwords en mi lista (2): 609


['a',
 'actualmente',
 'adelante',
 'además',
 'afirmó',
 'agregó',
 'ahora',
 'ahí',
 'al',
 'algo',
 'alguna',
 'algunas',
 'alguno',
 'algunos',
 'algún']

In [119]:
# (3) Uso de NLP Toolkit library

## Para instalar la librería revisa la página: https://www.nltk.org/data.html
## Nota, este proceso puede ser tardado...
# !pip install nltk
# nltk.download()

# load library
import nltk

stopwords_3 = nltk.corpus.stopwords.words('spanish')

print("Número de stopwords en de NLTK library (3):", len(stopwords_3))
stopwords_3[:15]

Número de stopwords en de NLTK library (3): 313


['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con']

**Vamos a eliminar las stopwords usando la lista que obtivimos de NLTK library (opción 3):**

In [120]:
#defining the function to remove stopwords from tokenized text
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords_3]
    return output

#applying the stopwords function to thet tweets
df['reseña_limpia'] = df['reseña_limpia'].apply(lambda x:remove_stopwords(x))
df.head()

,reseña,reseña_limpia
0,Uno de los otros críticos ha mencionado que de...,"[críticos, mencionado, después, ver, solo, 1, ..."
1,Una pequeña pequeña producción.La técnica de f...,"[pequeña, pequeña, producciónla, técnica, film..."
2,Pensé que esta era una manera maravillosa de p...,"[pensé, manera, maravillosa, pasar, tiempo, fi..."
3,"Básicamente, hay una familia donde un niño peq...","[básicamente, familia, niño, pequeño, jake, pi..."
4,"El ""amor en el tiempo"" de Petter Mattei es una...","[amor, tiempo, petter, mattei, película, visua..."


### Lematización
Este paso nos servirá para extraer la la raíz de las palabras, para poder "normalizar" el texto y realizar un mejor análisis. Aquí nuevamente utilizaremos la librería `NLTK`.

In [121]:
%%time

#defining the object for Lemmatization
wordnet_lemmatizer = nltk.stem.WordNetLemmatizer()

#defining the function for lemmatization
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text

#lemmatization of our tweets
df['reseña_limpia'] = df['reseña_limpia'].apply(lambda x:lemmatizer(x))
df.head()

CPU times: user 26.8 s, sys: 829 ms, total: 27.6 s
Wall time: 28 s


,reseña,reseña_limpia
0,Uno de los otros críticos ha mencionado que de...,"[críticos, mencionado, después, ver, solo, 1, ..."
1,Una pequeña pequeña producción.La técnica de f...,"[pequeña, pequeña, producciónla, técnica, film..."
2,Pensé que esta era una manera maravillosa de p...,"[pensé, manera, maravillosa, pasar, tiempo, fi..."
3,"Básicamente, hay una familia donde un niño peq...","[básicamente, familia, niño, pequeño, jake, pi..."
4,"El ""amor en el tiempo"" de Petter Mattei es una...","[amor, tiempo, petter, mattei, película, visua..."


### Eliminar URLs
Si quisiera eliminar URLs, podríamos usar lo siguiente:

In [122]:
def erase_urls(text):
    output = [i for i in text if i.rfind("http")==-1]
    return output

# texto.rfind("villano")
df['reseña_limpia'] = df['reseña_limpia'].apply(lambda x:erase_urls(x))
df.head()

,reseña,reseña_limpia
0,Uno de los otros críticos ha mencionado que de...,"[críticos, mencionado, después, ver, solo, 1, ..."
1,Una pequeña pequeña producción.La técnica de f...,"[pequeña, pequeña, producciónla, técnica, film..."
2,Pensé que esta era una manera maravillosa de p...,"[pensé, manera, maravillosa, pasar, tiempo, fi..."
3,"Básicamente, hay una familia donde un niño peq...","[básicamente, familia, niño, pequeño, jake, pi..."
4,"El ""amor en el tiempo"" de Petter Mattei es una...","[amor, tiempo, petter, mattei, película, visua..."


### Resultado final de preproceso (BD final)
A continuación se muestran los textos iniciales en la columna "reseña", y la información preprocesada en la columna "reseña_limpia", a manera de comparación.

In [123]:
df[['reseña', 'reseña_limpia']]

,reseña,reseña_limpia
0,Uno de los otros críticos ha mencionado que de...,"[críticos, mencionado, después, ver, solo, 1, ..."
1,Una pequeña pequeña producción.La técnica de f...,"[pequeña, pequeña, producciónla, técnica, film..."
2,Pensé que esta era una manera maravillosa de p...,"[pensé, manera, maravillosa, pasar, tiempo, fi..."
3,"Básicamente, hay una familia donde un niño peq...","[básicamente, familia, niño, pequeño, jake, pi..."
4,"El ""amor en el tiempo"" de Petter Mattei es una...","[amor, tiempo, petter, mattei, película, visua..."
...,...,...
49995,Pensé que esta película hizo un buen trabajo a...,"[pensé, película, hizo, buen, trabajo, derecha..."
49996,"Mala parcela, mal diálogo, mala actuación, dir...","[mala, parcela, mal, diálogo, mala, actuación,..."
49997,Soy católica enseñada en escuelas primarias pa...,"[católica, enseñada, escuelas, primarias, parr..."
49998,Voy a tener que estar en desacuerdo con el com...,"[voy, tener, desacuerdo, comentario, anterior,..."


In [124]:
print("Ejemplo de reseña...\n- original:\n",df['reseña'][1],
      "\n\n- reseña limpia:\n", df['reseña_limpia'][1])

Ejemplo de reseña...
- original:
 Una pequeña pequeña producción.La técnica de filmación es muy incuestionable, muy antigua, la moda de la BBC y le da una sensación de realismo reconfortante, y, a veces, incómodo, y, a veces, a la pieza.Los actores son extremadamente bien elegidos, Michael Sheen, no solo "tiene todo el polari", ¡pero tiene todas las voces por palmaditas!Realmente puede ver la edición perfecta guiada por las referencias a las entradas del diario de Williams, no solo vale la pena la observación, pero es una pieza imperrementemente escrita y realizada.Una producción magistral sobre uno de los grandes maestros de la comedia y su vida.El realismo realmente llega a casa con las pequeñas cosas: la fantasía del guardia que, en lugar de usar las técnicas de "sueño" tradicionales permanece sólido, entonces desaparece.Se desempeña nuestro conocimiento y nuestros sentidos, particularmente con las escenas relacionadas con Orton y Halliwell y los conjuntos (particularmente de su apa

## 3. Regular expressions (*Regex*)
Las expresiones regulares mejor conocidas como *regex* son una herramienta que nos sirve para trabajar con texto, con el cual se pueden realizar distintas operaciones como búsqueda o reemplazo de un texto específico.

Existen distintos operadores que se utilizan en *regex*, estos son los que utilizaremos para nuestros ejemplos. 


|Operador|Descripción (_en inglés_)|
|:------:|:----------------------|
|. (punto)|In the default mode, this matches any character except a newline. If the DOTALL flag has been specified, this matches any character including a newline.|
|* (asterisco) |Causes the resulting RE to match 0 or more repetitions of the preceding RE, as many repetitions as are possible. ab* will match ‘a’, ‘ab’, or ‘a’ followed by any number of ‘b’s.|
|+ (más) |Causes the resulting RE to match 1 or more repetitions of the preceding RE. ab+ will match ‘a’ followed by any non-zero number of ‘b’s; it will not match just ‘a’.|
|[ ]| Used to indicate a set of characters. In a set:<ul><li>Characters can be listed individually, e.g. [amk] will match 'a', 'm', or 'k'.</li><li>Ranges of characters can be indicated by giving two characters and separating them by a '-', for example [a-z] will match any lowercase ASCII letter, [0-5][0-9] will match all the two-digits numbers from 00 to 59, and [0-9A-Fa-f] will match any hexadecimal digit. If - is escaped (e.g. [a\-z]) or if it’s placed as the first or last character (e.g. [-a] or [a-]), it will match a literal '-'.</li><li>Special characters lose their special meaning inside sets. For example, [(+*)] will match any of the literal characters '(', '+', '*', or ')'.</li></ul>|
|\n |line break|
|\s|space|

⚠️ Se recomienda ampliamente que revisen la página de la [librería **`re`** - *Regular expression operations*](https://docs.python.org/3/library/re.html) para identificar todos los operadores, documentación y ejemplos de como utilizar _regex_.

📌 Si les interesa practicar el uso de _regex_, les recomiendo la página de [regexr.com](https://regexr.com)

In [127]:
# load regex library
import re

In [133]:
# Extracto de canción "El amor" de Arjona, link: https://www.letras.com/arjona-ricardo/1963651/
texto = """El amor tiene firma de autor
En las causas pérdidas
El amor siempre empieza soñando
Y termina en insomnio
Es un acto profundo de fe
Que huele a mentira
El amor baila al son que le toquen
Sea Dios o el demonio
Sea Dios o el demonio

El amor es la guerra pérdida
Entre el sexo y la risa
Es la llave con que abres
El grifo del agua en los ojos
Es el tiempo más lento del mundo
Cuando va de prisa
El amor se abre paso despacio
No importa el cerrojo

El AmOr es la arrogancia
De aferrarse a lo imposible
Es buscar en otra parte
Lo que no encuentras en ti"""

### Encontrar un texto usando _regex_

In [134]:
# Encontrar todos los enunciados que tengan la palabra 'amor'
re.findall(r'.*[Aa][Mm][Oo][Rr].*', texto)

['El amor tiene firma de autor',
 'El amor siempre empieza soñando',
 'El amor baila al son que le toquen',
 'El amor es la guerra pérdida',
 'El amor se abre paso despacio',
 'El AmOr es la arrogancia']

#### Sustituir una palabra, frase o cadena de caracterés por otro texto

In [135]:
# Substituir la palabra 'amor' por 'cepillo de dientes'
texto_sub = re.sub('[Aa][Mm][Oo][Rr]', 'Guacamole', texto)  # cambia la palabra
texto_sub = re.sub('\n+', '. ', texto_sub)  # cambia el salto de dos renglones por un punto 
texto_sub

'El Guacamole tiene firma de autor. En las causas pérdidas. El Guacamole siempre empieza soñando. Y termina en insomnio. Es un acto profundo de fe. Que huele a mentira. El Guacamole baila al son que le toquen. Sea Dios o el demonio. Sea Dios o el demonio. El Guacamole es la guerra pérdida. Entre el sexo y la risa. Es la llave con que abres. El grifo del agua en los ojos. Es el tiempo más lento del mundo. Cuando va de prisa. El Guacamole se abre paso despacio. No importa el cerrojo. El Guacamole es la arrogancia. De aferrarse a lo imposible. Es buscar en otra parte. Lo que no encuentras en ti'

### Dividir el texto en una lista de strings

In [136]:
texto # original

'El amor tiene firma de autor\nEn las causas pérdidas\nEl amor siempre empieza soñando\nY termina en insomnio\nEs un acto profundo de fe\nQue huele a mentira\nEl amor baila al son que le toquen\nSea Dios o el demonio\nSea Dios o el demonio\n\nEl amor es la guerra pérdida\nEntre el sexo y la risa\nEs la llave con que abres\nEl grifo del agua en los ojos\nEs el tiempo más lento del mundo\nCuando va de prisa\nEl amor se abre paso despacio\nNo importa el cerrojo\n\nEl AmOr es la arrogancia\nDe aferrarse a lo imposible\nEs buscar en otra parte\nLo que no encuentras en ti'

In [76]:
re.split("\n+", texto)

['El amor tiene firma de autor',
 'En las causas pérdidas',
 'El amor siempre empieza soñando',
 'Y termina en insomnio',
 'Es un acto profundo de fe',
 'Que huele a mentira',
 'El amor baila al son que le toquen',
 'Sea Dios o el demonio',
 'Sea Dios o el demonio',
 'El amor es la guerra pérdida',
 'Entre el sexo y la risa',
 'Es la llave con que abres',
 'El grifo del agua en los ojos',
 'Es el tiempo más lento del mundo',
 'Cuando va de prisa',
 'El amor se abre paso despacio',
 'No importa el cerrojo',
 'El amor es la arrogancia',
 'De aferrarse a lo imposible',
 'Es buscar en otra parte',
 'Lo que no encuentras en ti']

## 4. spaCy

In [137]:
# # Install spaCy webpage: https://spacy.io/usage
# ! conda install -c conda-forge spacy  # <-install spaCy
# ! python -m spacy download en_core_web_sm   # <-spaCy English pipeline optimized for CPU.
# ! python -m spacy download es_core_news_sm  # <-spaCy Spanish pipeline optimized for CPU.

In [138]:
# load library
import spacy

### Obtener características de cada *token*

In [139]:
# load spaCy Spanish pipeline optimized
nlp = spacy.load("es_core_news_sm")

# ejemplo
enunciado = 'Me llegó un email al correo leo234@gmail.com que dice: "OMG! El señor Carlos rompió el control\
del televisor hoy y tuve que comprar otro en www.amazon.com.mx".'

# volver el enunciado un objeto tipo NLP
doc = nlp(enunciado) #(sentences[0])
print("Enunciado:\n",doc.text,"\n")

# Para cada token (palabra/cadena) se obtienen distintas características:
# - texto
# - tag con tipo de texto: noun, verb, determinant, adverb, conjunction
# - dep: dependenxy
# - like_email: identifica si el token es un email
# - like_url: identifica si el token es una página web
list_tokens = []
for token in doc:
    list_tokens.append([token.text,token.lemma_,token.pos_,token.dep_,token.like_email,
                        token.like_url, token.head.text])

# Revisar algunas de las características de cada token
pd.DataFrame(list_tokens, columns=['text','lemma','pos','dep','is_email','is_url','next_text'])

/Users/vcuspinera/opt/miniconda3/lib/python3.10/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'es_core_news_sm' (3.3.0) was trained with spaCy v3.3.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Enunciado:
 Me llegó un email al correo leo234@gmail.com que dice: "OMG! El señor Carlos rompió el controldel televisor hoy y tuve que comprar otro en www.amazon.com.mx". 



,text,lemma,pos,dep,is_email,is_url,next_text
0,Me,yo,PRON,iobj,False,False,llegó
1,llegó,llegar,VERB,ROOT,False,False,llegó
2,un,uno,DET,det,False,False,email
3,email,email,NOUN,nsubj,False,False,llegó
4,al,al,ADP,case,False,False,correo
5,correo,correo,NOUN,obl,False,False,llegó
6,leo234@gmail.com,leo234@gmail.com,NUM,appos,True,False,correo
7,que,que,PRON,nsubj,False,False,dice
8,dice,decir,VERB,acl,False,False,correo
9,:,:,PUNCT,punct,False,False,OMG


### Preproceso con spaCy
Ejemplo de función para preproceso usando spaCy, Regex y métodos de strings, comparando resultados con el del preproceso que se vio al inicio de este Jupyter notebook.

In [148]:
def preprocess(text,
               min_token_len = 2,
               irrelevant_pos = ['PRON', 'SPACE', 'PUNCT', 'ADV', 'ADP', 'CCONJ', 'AUX', 'PRP', 'DET'],
               avoid_entities = ['PERSON', 'ORG', 'LOC', 'GPE', 'DET']):
    # note: Didn't use the following options in the `preprocess_comments`
    #    - 'PROPN', erase proper names, but also words as orange.
    #    - 'DET', removes the word 'no', which changes the meaning.
    """
    Function that identify sensible information, anonymize and transforms
    the data in a useful format for using with tokens.
    Parameters
    -------------
    text : (list)
        the list of text to be preprocessed
    irrelevant_pos : (list)
        a list of irrelevant 'pos' tags
    avoid_entities : (list)
        a list of entity labels to be avoided

    Returns
    -------------
    (list) list of preprocessed text

    Example
    -------------
    example = ["Hello, I'm George and I love swimming!",
                "I am a really good cook; what about you?",
                "Contact me at george23@gmail.com"]
    preprocess(example)
    (output:) ['hello love swimming', 'good cook', 'contact']
    """
    result = []
    others = ["'s", "the", "that", "this", "to", "-PRON-"]
    # comment: "-PRON-" is a lemma for "my", "your", etc.

    # function
    for sent in text:
        # <string methods>
        sent = str(sent).lower()
        
        # <regex>
        sent = re.sub(r"(F|f)acebook", "redes sociales", sent)
        sent = re.sub(r"(T|t)witter", "redes sociales", sent)
        sent = re.sub(r"(I|i)nstagram", "redes sociales", sent)
        
        result_sent = []
        
        # <spaCy>
        doc = nlp(sent)
        entities = [str(ent) for ent in doc.ents if ent.label_ in avoid_entities]
                   # This helps to detect names of persons, organization and dates
        for token in doc:            
            if (token.like_email or
                token.like_url or
                token.pos_ in irrelevant_pos or
                str(token) in entities or
                str(token.lemma_) in others or
                len(token) < min_token_len):
                continue
            else:
                result_sent.append(token.lemma_)
        result.append(result_sent)
    return result

In [149]:
# Seleccionamos una parte del dataframe para correr el preproceso e n esta BD corta
df_corta = df[:200]
df_corta

,reseña,reseña_limpia
0,Uno de los otros críticos ha mencionado que de...,"[críticos, mencionado, después, ver, solo, 1, ..."
1,Una pequeña pequeña producción.La técnica de f...,"[pequeña, pequeña, producciónla, técnica, film..."
2,Pensé que esta era una manera maravillosa de p...,"[pensé, manera, maravillosa, pasar, tiempo, fi..."
3,"Básicamente, hay una familia donde un niño peq...","[básicamente, familia, niño, pequeño, jake, pi..."
4,"El ""amor en el tiempo"" de Petter Mattei es una...","[amor, tiempo, petter, mattei, película, visua..."
...,...,...
195,Fantasma .... Clase.Phantasmo II ..... increíb...,"[fantasma, clasephantasmo, ii, increíblephanta..."
196,Ridículo. Annakin angelical de 9 años se convi...,"[ridículo, annakin, angelical, 9, años, convie..."
197,"Scotty (Grant Cramer, quien continuaría en la ...","[scotty, grant, cramer, continuaría, gran, mov..."
198,Si mantienes una perspectiva histórica rígida ...,"[si, mantienes, perspectiva, histórica, rígida..."


In [150]:
%%time

# Preprocess tweets
df_corta['reseña_spacy_preproceso'] = preprocess(df_corta['reseña'])

CPU times: user 9.37 s, sys: 1.31 s, total: 10.7 s
Wall time: 11 s


<timed exec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [151]:
df_corta

,reseña,reseña_limpia,reseña_spacy_preproceso
0,Uno de los otros críticos ha mencionado que de...,"[críticos, mencionado, después, ver, solo, 1, ...","[crítico, mencionar, que, ver, episodio, engan..."
1,Una pequeña pequeña producción.La técnica de f...,"[pequeña, pequeña, producciónla, técnica, film...","[pequeño, pequeño, técnico, filmación, incuest..."
2,Pensé que esta era una manera maravillosa de p...,"[pensé, manera, maravillosa, pasar, tiempo, fi...","[pensar, que, manera, maravilloso, pasar, tiem..."
3,"Básicamente, hay una familia donde un niño peq...","[básicamente, familia, niño, pequeño, jake, pi...","[familia, niño, pequeño, jake, pensar, que, zo..."
4,"El ""amor en el tiempo"" de Petter Mattei es una...","[amor, tiempo, petter, mattei, película, visua...","[amor, tiempo, petter, mattei, película, visua..."
...,...,...,...
195,Fantasma .... Clase.Phantasmo II ..... increíb...,"[fantasma, clasephantasmo, ii, increíblephanta...","[fantasma, iii, erm, terrible, aunque, encanta..."
196,Ridículo. Annakin angelical de 9 años se convi...,"[ridículo, annakin, angelical, 9, años, convie...","[ridículo, annakin, angelical, año, convertir,..."
197,"Scotty (Grant Cramer, quien continuaría en la ...","[scotty, grant, cramer, continuaría, gran, mov...","[scotty, grant, cramer, continuar, gran, movie..."
198,Si mantienes una perspectiva histórica rígida ...,"[si, mantienes, perspectiva, histórica, rígida...","[si, mantener, perspectiva, histórico, rígido,..."


## 5. Referencias
- [Python String Methods](https://docs.python.org/3/library/stdtypes.html) in Python Documentation.
- [Text Preprocessing in NLP with Python codes](https://www.analyticsvidhya.com/blog/2021/06/text-preprocessing-in-nlp-with-python-codes/) by Deepanshi, in Analytics Vidhya.
- Librería de Python [*Natural Language Toolkit* - `NLTK`](https://www.nltk.org/index.html)
- Librería de Python [*Regular expression operations* - `re`](https://docs.python.org/3/library/re.html)
- Material público del curso [DSCI 575: Advanced Machine Learning](https://github.com/UBC-MDS/DSCI_575_adv-mach-learn) de UBC MDS.
- Python script [preprocess.py](https://github.com/vcuspinera/UBC_MDS_Capstone_BCStats/blob/master/src/data/preprocess.py) del Capstone project de UBC MDS y BC Stats, por Sukriti Trehan, Karanpal Singh, Carlina Kim y Victor Cuspinera.